This notebook deals with the description of ICD codes and their conversion to ICD10 codes from tables of MIMIC III database. Both types of codes are described:
- [ICD9 Diagnosis codes](#icd9)
- [ICD9 Procedures codes](#icdpcs)
> The working aim and summaries are mentioned in the [google slide](https://docs.google.com/presentation/d/19tE0L4fJ9JjG6qf05m8iwDG0PH1cGIKFhiyBTqZ6Ofk/edit?usp=sharing)<br>
> The work with text and extracting relevant sections are in this [notebook](./Discharge%20summary.ipynb)

In [2]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_colwidth', None)

# <a id='icd9'>ICD9 Diagnosis Codes</a>
Tables used are:
- ADMISSIONS.csv: This has all patient records of the hospital
- D_ICD_DIAGNOSES.csv: This contain the ICD9 diagnosi code and definitions
- DIAGNOSES_ICD.csv: contains ICD codes corresponting to different admission ids

In [3]:
patient=pd.read_csv('Data MIMIC III\ADMISSIONS.csv')
patient.head()


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT/SDA,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [4]:
icd=pd.read_csv('Data MIMIC III/D_ICD_DIAGNOSES.csv')
icd.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bacilli not found by bacteriological or histological examination, but tuberculosis confirmed by other methods [inoculation of animals]"
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or histological examination not done"
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or histological examination unknown (at present)"
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli found (in sputum) by microscopy"


In [5]:
diag=pd.read_csv('Data MIMIC III/DIAGNOSES_ICD.csv')
diag.head()


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [76]:
print("Number of Admission IDs that are null %d" %diag[diag.HADM_ID.isnull()].ROW_ID.count())
print("Number of Admission Ids that dont have ICD9 codes %d" %diag[diag.ICD9_CODE.isnull()].HADM_ID.count())

Number of Admission IDs that are null 0
Number of Admission Ids that dont have ICD9 codes 47


In [7]:
print("Total number of unique patients is %d" %diag.SUBJECT_ID.nunique())
print("Total number of unique patient admissions is %d " %diag.HADM_ID.nunique())
print("Total number of unique ICD codes is %d" %diag.ICD9_CODE.nunique())
print("Total number of admissions assigned ICD codes is %d " %diag[diag.ICD9_CODE.notnull()].HADM_ID.nunique())

Total number of unique patients is 46520
Total number of unique patient admissions is 58976 
Total number of unique ICD codes is 6984
Total number of admissions assigned ICD codes is 58929 


In [78]:
diag.dropna(subset=["ICD9_CODE"], inplace=True) #Dropped all those admiss.Id that don't contain ICD codes
print("Total number of unique patient admissions is %d " %diag.HADM_ID.nunique())

Total number of unique patient admissions is 58929 


In [79]:
icd_freq=diag.groupby('ICD9_CODE').ICD9_CODE.count().to_frame()
icd_freq.rename(columns={icd_freq.columns[0]:'count'},inplace=True)
icd_freq.reset_index(level=0,inplace=True)
icd_freq


,ICD9_CODE,count
0,0030,2
1,0031,1
2,0038,2
3,0039,1
4,0041,3
...,...,...
6979,V902,1
6980,V9039,1
6981,V9081,3
6982,V9089,1


In [9]:
icd_freq.sort_values(by='count',ascending=False).head(100).reset_index(drop=True)                                 

,ICD9_CODE,count
0,4019,20703
1,4280,13111
2,42731,12891
3,41401,12429
4,5849,9119
...,...,...
95,42833,1220
96,07054,1218
97,42732,1217
98,E8782,1211


In [10]:
icd[icd.ICD9_CODE=='2800']

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
2494,2218,2800,Chr blood loss anemia,Iron deficiency anemia secondary to blood loss (chronic)


> The below dictionary of ICD9 codes to ICD10 codes is made through <http://www.icd10codesearch.com/>

In [11]:
icd10_dict={'ICD9':['4019','4280','42731','41401','5849','25000','2724','2724','51881','51881','5990','53081','2720','V053','V290','2859','2449','486','2851','2762','496','99592','V5861','0389','5070','V3000','5859','311','40390','3051','412','2875','V4581','41071','2761','4240','4240','V1582','V3001','5119','V4582','40391','78552','4241','4241','4241','4241','4241','V5867','42789','42789','32723','9971','9971','9971','9971','5845','2760','7742','49390','49390','2767','5180','5180','4168','4168','45829','45829','2749','4589','V502'
                   ,'73300','78039','5856','4271','4254','4254','4111','V1251','3572','3572','3572','3572','3572','30000','27800','E8798','41400','60000','7907','2930','00845','2768','4439','5789','V4501','27651','28521','27652','431','4275','E8788','V4986','769','79902','70703','70703','70703','5715','5715','5715','V103','99591','2639','42832','42833','07054','42732','E8782','V1046'],
            'ICD10':['I10','I509','I4891','I2510','N179','E119','E784','E785','J9600','J9690','N390','K219','E780','Z23','P002','D649','E039','J189','D62','E872','J449','R6520','Z7901','A419','J690','Z3800','N189','F329','I129','F17200','I252','D696','Z951','I214','E871','I340','I348','Z87891','Z3801','J918','Z9861','I120','R6521','I350','I351','I352','I358','I359','Z794','I498','R001','G4733','I97710','I97790','I9788','I9789','N170','E870','P590','J45909','J45998','E875','J9811','J9819','I272','I2789','I952','I9581','M109','I959','Z412'
                    ,'M810','R569','N186','I472','I425','I428','I200','Z86718','E0842','E0942','E1042','E1142','E1342','F419','E669','Y848','I2510','N400','R7881','F05','A047','E876','I739','K922','Z950','E860','D631','E861','I619','I469','Y838','Z66','P220','R0902','L89139','L89149','L89159','K740','K7460','K7469','Z853','A419','E46','I5032','I5033','B182','I4892','Y832','Z8546'],
            'long_title':['ESSENTIAL (PRIMARY) HYPERTENSION','HEART FAILURE, UNSPECIFIED','UNSPECIFIED ATRIAL FIBRILLATION','ATHEROSCLEROTIC HEART DISEASE OF NATIVE CORONARY ARTERY WITHOUT ANGINA PECTORIS'
                         ,'ACUTE KIDNEY FAILURE, UNSPECIFIED','TYPE 2 DIABETES MELLITUS WITHOUT COMPLICATIONS','OTHER HYPERLIPIDEMIA','HYPERLIPIDEMIA, UNSPECIFIED',
                         'ACUTE RESPIRATORY FAILURE, UNSPECIFIED WHETHER WITH HYPOXIA OR HYPERCAPNIA','RESPIRATORY FAILURE, UNSPECIFIED, UNSPECIFIED WHETHER WITH HYPOXIA OR HYPERCAPNIA'
                         ,'URINARY TRACT INFECTION, SITE NOT SPECIFIED','GASTRO-ESOPHAGEAL REFLUX DISEASE WITHOUT ESOPHAGITIS','PURE HYPERCHOLESTEROLEMIA','ENCOUNTER FOR IMMUNIZATION'
                         ,'NEWBORN (SUSPECTED TO BE) AFFECTED BY MATERNAL INFECTIOUS AND PARASITIC DISEASES','ANEMIA, UNSPECIFIED','HYPOTHYROIDISM, UNSPECIFIED','PNEUMONIA, UNSPECIFIED ORGANISM'
                         ,'ACUTE POSTHEMORRHAGIC ANEMIA','ACIDOSIS','CHRONIC OBSTRUCTIVE PULMONARY DISEASE, UNSPECIFIED','SEVERE SEPSIS WITHOUT SEPTIC SHOCK','LONG TERM (CURRENT) USE OF ANTICOAGULANTS'
                         ,'SEPSIS, UNSPECIFIED ORGANISM','PNEUMONITIS DUE TO INHALATION OF FOOD AND VOMIT','SINGLE LIVEBORN INFANT, DELIVERED VAGINALLY','CHRONIC KIDNEY DISEASE, UNSPECIFIED','MAJOR DEPRESSIVE DISORDER, SINGLE EPISODE, UNSPECIFIED'
                         ,'HYPERTENSIVE CHRONIC KIDNEY DISEASE WITH STAGE 1 THROUGH STAGE 4 CHRONIC KIDNEY DISEASE, OR UNSPECIFIED CHRONIC KIDNEY DISEASE','NICOTINE DEPENDENCE, UNSPECIFIED, UNCOMPLICATED','OLD MYOCARDIAL INFARCTION','THROMBOCYTOPENIA, UNSPECIFIED'
                         ,'PRESENCE OF AORTOCORONARY BYPASS GRAFT','NON-ST ELEVATION (NSTEMI) MYOCARDIAL INFARCTION','HYPO-OSMOLALITY AND HYPONATREMIA','NONRHEUMATIC MITRAL (VALVE) INSUFFICIENCY','OTHER NONRHEUMATIC MITRAL VALVE DISORDERS'
                         ,'PERSONAL HISTORY OF NICOTINE DEPENDENCE','SINGLE LIVEBORN INFANT, DELIVERED BY CESAREAN','PLEURAL EFFUSION IN OTHER CONDITIONS CLASSIFIED ELSEWHERE','CORONARY ANGIOPLASTY STATUS','HYPERTENSIVE CHRONIC KIDNEY DISEASE WITH STAGE 5 CHRONIC KIDNEY DISEASE OR END STAGE RENAL DISEASE','SEVERE SEPSIS WITH SEPTIC SHOCK'
                         ,'NONRHEUMATIC AORTIC (VALVE) STENOSIS','NONRHEUMATIC AORTIC (VALVE) INSUFFICIENCY','NONRHEUMATIC AORTIC (VALVE) STENOSIS WITH INSUFFICIENCY','OTHER NONRHEUMATIC AORTIC VALVE DISORDERS','ONRHEUMATIC AORTIC VALVE DISORDER, UNSPECIFIED','LONG TERM (CURRENT) USE OF INSULIN','OTHER SPECIFIED CARDIAC ARRHYTHMIAS','BRADYCARDIA, UNSPECIFIED'
                         ,'OBSTRUCTIVE SLEEP APNEA (ADULT) (PEDIATRIC)','INTRAOPERATIVE CARDIAC ARREST DURING CARDIAC SURGERY','OTHER INTRAOPERATIVE CARDIAC FUNCTIONAL DISTURBANCES DURING CARDIAC SURGERY','OTHER INTRAOPERATIVE COMPLICATIONS OF THE CIRCULATORY SYSTEM, NOT ELSEWHERE CLASSIFIED','OTHER POSTPROCEDURAL COMPLICATIONS AND DISORDERS OF THE CIRCULATORY SYSTEM, NOT ELSEWHERE CLASSIFIED'
                         ,'ACUTE KIDNEY FAILURE WITH TUBULAR NECROSIS','HYPEROSMOLALITY AND HYPERNATREMIA','NEONATAL JAUNDICE ASSOCIATED WITH PRETERM DELIVERY','UNSPECIFIED ASTHMA, UNCOMPLICATED','OTHER ASTHMA','HYPERKALEMIA','ATELECTASIS','OTHER PULMONARY COLLAPSE','OTHER SECONDARY PULMONARY HYPERTENSION','OTHER SPECIFIED PULMONARY HEART DISEASES'
                         ,'HYPOTENSION DUE TO DRUGS','POSTPROCEDURAL HYPOTENSION','GOUT, UNSPECIFIED','HYPOTENSION, UNSPECIFIED','ENCOUNTER FOR ROUTINE AND RITUAL MALE CIRCUMCISION'
                         ,'AGE-RELATED OSTEOPOROSIS WITHOUT CURRENT PATHOLOGICAL FRACTURE','UNSPECIFIED CONVULSIONS','END STAGE RENAL DISEASE','VENTRICULAR TACHYCARDIA','OTHER RESTRICTIVE CARDIOMYOPATHY','OTHER CARDIOMYOPATHIES','UNSTABLE ANGINA','PERSONAL HISTORY OF OTHER VENOUS THROMBOSIS AND EMBOLISM'
                         ,'DIABETES MELLITUS DUE TO UNDERLYING CONDITION WITH DIABETIC POLYNEUROPATHY','DRUG OR CHEMICAL INDUCED DIABETES MELLITUS WITH NEUROLOGICAL COMPLICATIONS WITH DIABETIC POLYNEUROPATHY','TYPE 1 DIABETES MELLITUS WITH DIABETIC POLYNEUROPATHY','TYPE 2 DIABETES MELLITUS WITH DIABETIC POLYNEUROPATHY','OTHER SPECIFIED DIABETES MELLITUS WITH DIABETIC POLYNEUROPATHY'
                         ,'ANXIETY DISORDER, UNSPECIFIED','OBESITY, UNSPECIFIED','OTHER MEDICAL PROCEDURES AS THE CAUSE OF ABNORMAL REACTION OF THE PATIENT, OR OF LATER COMPLICATION, WITHOUT MENTION OF MISADVENTURE AT THE TIME OF THE PROCEDURE','ATHEROSCLEROTIC HEART DISEASE OF NATIVE CORONARY ARTERY WITHOUT ANGINA PECTORIS'
                         ,'ENLARGED PROSTATE WITHOUT LOWER URINARY TRACT SYMPTOMS','BACTEREMIA','DELIRIUM DUE TO KNOWN PHYSIOLOGICAL CONDITION','ENTEROCOLITIS DUE TO CLOSTRIDIUM DIFFICILE','HYPOKALEMIA','PERIPHERAL VASCULAR DISEASE, UNSPECIFIED','GASTROINTESTINAL HEMORRHAGE, UNSPECIFIED','PRESENCE OF CARDIAC PACEMAKER','DEHYDRATION','ANEMIA IN CHRONIC KIDNEY DISEASE'
                         ,'HYPOVOLEMIA','NONTRAUMATIC INTRACEREBRAL HEMORRHAGE, UNSPECIFIED','CARDIAC ARREST, CAUSE UNSPECIFIED','OTHER SURGICAL PROCEDURES AS THE CAUSE OF ABNORMAL REACTION OF THE PATIENT, OR OF LATER COMPLICATION, WITHOUT MENTION OF MISADVENTURE AT THE TIME OF THE PROCEDURE','DO NOT RESUSCITATE','RESPIRATORY DISTRESS SYNDROME OF NEWBORN','HYPOXEMIA',
                          'PRESSURE ULCER OF RIGHT LOWER BACK, UNSPECIFIED STAGE','PRESSURE ULCER OF LEFT LOWER BACK, UNSPECIFIED STAGE','PRESSURE ULCER OF SACRAL REGION, UNSPECIFIED STAGE','HEPATIC FIBROSIS','UNSPECIFIED CIRRHOSIS OF LIVER','OTHER CIRRHOSIS OF LIVER','PERSONAL HISTORY OF MALIGNANT NEOPLASM OF BREAST','SEPSIS, UNSPECIFIED ORGANISM','UNSPECIFIED PROTEIN-CALORIE MALNUTRITION','CHRONIC DIASTOLIC (CONGESTIVE) HEART FAILURE'
                         ,'ACUTE ON CHRONIC DIASTOLIC (CONGESTIVE) HEART FAILURE','CHRONIC VIRAL HEPATITIS C','UNSPECIFIED ATRIAL FLUTTER','SURGICAL OPERATION WITH ANASTOMOSIS, BYPASS OR GRAFT AS THE CAUSE OF ABNORMAL REACTION OF THE PATIENT, OR OF LATER COMPLICATION, WITHOUT MENTION OF MISADVENTURE AT THE TIME OF THE PROCEDURE','PERSONAL HISTORY OF MALIGNANT NEOPLASM OF PROSTATE']}

In [12]:
icd10= pd.DataFrame.from_dict(icd10_dict)

In [13]:
icd10.rename(columns={icd10.columns[0]:'ICD9_CODE'},inplace=True)
icd10.columns


Index(['ICD9_CODE', 'ICD10', 'long_title'], dtype='object')

In [14]:
icd10


,ICD9_CODE,ICD10,long_title
0,4019,I10,ESSENTIAL (PRIMARY) HYPERTENSION
1,4280,I509,"HEART FAILURE, UNSPECIFIED"
2,42731,I4891,UNSPECIFIED ATRIAL FIBRILLATION
3,41401,I2510,ATHEROSCLEROTIC HEART DISEASE OF NATIVE CORONARY ARTERY WITHOUT ANGINA PECTORIS
4,5849,N179,"ACUTE KIDNEY FAILURE, UNSPECIFIED"
...,...,...,...
115,42833,I5033,ACUTE ON CHRONIC DIASTOLIC (CONGESTIVE) HEART FAILURE
116,07054,B182,CHRONIC VIRAL HEPATITIS C
117,42732,I4892,UNSPECIFIED ATRIAL FLUTTER
118,E8782,Y832,"SURGICAL OPERATION WITH ANASTOMOSIS, BYPASS OR GRAFT AS THE CAUSE OF ABNORMAL REACTION OF THE PATIENT, OR OF LATER COMPLICATION, WITHOUT MENTION OF MISADVENTURE AT THE TIME OF THE PROCEDURE"


In [15]:
frequently=icd_freq.sort_values(by='count',ascending=False).head(100)
listoffrequent=list(frequently.ICD9_CODE)
print(listoffrequent)

['4019', '4280', '42731', '41401', '5849', '25000', '2724', '51881', '5990', '53081', '2720', 'V053', 'V290', '2859', '2449', '486', '2851', '2762', '496', '99592', 'V5861', '0389', '5070', 'V3000', '5859', '311', '40390', '3051', '412', '2875', 'V4581', '41071', '2761', '4240', 'V1582', 'V3001', '5119', 'V4582', '40391', '78552', '4241', 'V5867', '42789', '32723', '9971', '5845', '2760', '7742', '49390', '2767', '5180', '4168', '45829', '2749', '4589', 'V502', '73300', '78039', '5856', '4271', '5185', '4254', '4111', 'V1251', '3572', '30000', '99811', '27800', 'E8798', '41400', '60000', '7907', '2930', '00845', '2768', '4439', '5789', 'V4501', '27651', '28521', '27652', '431', '4275', 'E8788', '2765', 'V4986', '769', '79902', '70703', '5715', 'V103', '99591', '2639', '42832', '99812', '42833', '07054', '42732', 'E8782', 'V1046']


> ICD10 codes that correspond to more than 1 ICD9 codes

In [16]:
icd10[icd10.duplicated(['ICD10'])] 


,ICD9_CODE,ICD10,long_title
87,41400,I2510,ATHEROSCLEROTIC HEART DISEASE OF NATIVE CORONARY ARTERY WITHOUT ANGINA PECTORIS
112,99591,A419,"SEPSIS, UNSPECIFIED ORGANISM"


In [17]:
icd10[(icd10.ICD10=='I2510') |(icd10.ICD10=='A419') ]

,ICD9_CODE,ICD10,long_title
3,41401,I2510,ATHEROSCLEROTIC HEART DISEASE OF NATIVE CORONARY ARTERY WITHOUT ANGINA PECTORIS
23,0389,A419,"SEPSIS, UNSPECIFIED ORGANISM"
87,41400,I2510,ATHEROSCLEROTIC HEART DISEASE OF NATIVE CORONARY ARTERY WITHOUT ANGINA PECTORIS
112,99591,A419,"SEPSIS, UNSPECIFIED ORGANISM"


> ICD( codes that have correspond to more than 5 ICD10 codes have been leftout

In [84]:
print("Number of unique ICD9 codes that have been converted to ICD10 :%d" %icd10.ICD9_CODE.nunique())
left_out=[item for item in listoffrequent if item not in icd10.ICD9_CODE.to_list()]
print(left_out)

['5185', '99811', '2765', '99812']
Number of unique ICD9 codes that have been converted to ICD10 :96


In [86]:
adm_icd=pd.merge(diag[['HADM_ID','ICD9_CODE']],icd10[['ICD9_CODE','ICD10']],how='left',on='ICD9_CODE')

In [87]:
adm_icd['ICD10']=adm_icd['ICD10'].fillna('Misc')
adm_icd

,HADM_ID,ICD9_CODE,ICD10
0,172335,40301,Misc
1,172335,486,J189
2,172335,58281,Misc
3,172335,5855,Misc
4,172335,4254,I425
...,...,...,...
711616,188195,20280,Misc
711617,188195,V5869,Misc
711618,188195,V1279,Misc
711619,188195,5275,Misc


In [89]:
adm_icd_agg=adm_icd.groupby('HADM_ID').agg({'ICD9_CODE':lambda x: ','.join(x),'ICD10':lambda y: ','.join(y)}).reset_index()

In [118]:
adm_icd_agg[(adm_icd_agg.HADM_ID==189797) |(adm_icd_agg.HADM_ID==100003)|(adm_icd_agg.HADM_ID==100006)|(adm_icd_agg.HADM_ID==100007) ].reset_index(drop=True)

,HADM_ID,ICD9_CODE,ICD10
0,100003,"53100,2851,07054,5715,5715,5715,45621,53789,4019,53550,7823","Misc,D62,B182,K740,K7460,K7469,Misc,Misc,I10,Misc,Misc"
1,100006,"49320,51881,51881,486,20300,2761,7850,3090,V1251,V1582","Misc,J9600,J9690,J189,Misc,E871,Misc,Misc,Z86718,Z87891"
2,100007,"56081,5570,9973,486,4019","Misc,Misc,Misc,J189,I10"
3,189797,"V3100,76518,7756,7706,V290,V053","Misc,Misc,Misc,Misc,P002,Z23"


In [91]:
adm_icd_agg

,HADM_ID,ICD9_CODE,ICD10
0,100001,"25013,3371,5849,5780,V5867,25063,5363,4580,25043,40390,5853,25053,36201,25083,7078,V1351","Misc,Misc,N179,Misc,Z794,Misc,Misc,Misc,Misc,I129,Misc,Misc,Misc,Misc,Misc,Misc"
1,100003,"53100,2851,07054,5715,5715,5715,45621,53789,4019,53550,7823","Misc,D62,B182,K740,K7460,K7469,Misc,Misc,I10,Misc,Misc"
2,100006,"49320,51881,51881,486,20300,2761,7850,3090,V1251,V1582","Misc,J9600,J9690,J189,Misc,E871,Misc,Misc,Z86718,Z87891"
3,100007,"56081,5570,9973,486,4019","Misc,Misc,Misc,J189,I10"
4,100009,"41401,99604,4142,25000,27800,V8535,4148,4111,V4582,V1582,2859,4019,2720,4262,44021,V153,V5867,V4502","I2510,Misc,Misc,E119,E669,Misc,Misc,I200,Z9861,Z87891,D649,I10,E780,Misc,Misc,Misc,Z794,Misc"
...,...,...,...
58924,199993,"41031,42821,42731,4271,5180,5180,4240,4240,2760,5119,5184","Misc,Misc,I4891,I472,J9811,J9819,I340,I348,E870,J918,Misc"
58925,199994,"486,4280,51881,51881,3970,496,4169,585,42732,2762","J189,I509,J9600,J9690,Misc,J449,Misc,Misc,I4892,E872"
58926,199995,"4210,7464,42971,30401,4412,44284,V1259,04111,30503,3051","Misc,Misc,Misc,Misc,Misc,Misc,Misc,Misc,Misc,F17200"
58927,199998,"41401,9971,9971,9971,9971,9975,42731,78820,4111,V4582,E8782,4293,4400,4019,4439,2724,2724,51889,2720,V4589","I2510,I97710,I97790,I9788,I9789,Misc,I4891,Misc,I200,Z9861,Y832,Misc,Misc,I10,I739,E784,E785,Misc,E780,Misc"


In [24]:
fields=['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'DESCRIPTION', 'ISERROR', 'TEXT']
discharge=pd.read_csv('Data MIMIC III/NOTEEVENTS.csv',usecols=fields)


In [25]:
discharge.head(1)

,SUBJECT_ID,HADM_ID,CATEGORY,DESCRIPTION,ISERROR,TEXT
0,22532,167853.0,Discharge summary,Report,NaN,"Admission Date: [**2151-7-16**] Discharge Date: [**2151-8-4**]\n\n\nService:\nADDENDUM:\n\nRADIOLOGIC STUDIES: Radiologic studies also included a chest\nCT, which confirmed cavitary lesions in the left lung apex\nconsistent with infectious process/tuberculosis. This also\nmoderate-sized left pleural effusion.\n\nHEAD CT: Head CT showed no intracranial hemorrhage or mass\neffect, but old infarction consistent with past medical\nhistory.\n\nABDOMINAL CT: Abdominal CT showed lesions of\nT10 and sacrum most likely secondary to osteoporosis. These can\nbe followed by repeat imaging as an outpatient.\n\n\n\n [**First Name8 (NamePattern2) **] [**First Name4 (NamePattern1) 1775**] [**Last Name (NamePattern1) **], M.D. [**MD Number(1) 1776**]\n\nDictated By:[**Hospital 1807**]\nMEDQUIST36\n\nD: [**2151-8-5**] 12:11\nT: [**2151-8-5**] 12:21\nJOB#: [**Job Number 1808**]\n"


In [26]:
discharge.CATEGORY.unique()

array(['Discharge summary', 'Echo', 'ECG', 'Nursing', 'Physician ',
       'Rehab Services', 'Case Management ', 'Respiratory ', 'Nutrition',
       'General', 'Social Work', 'Pharmacy', 'Consult', 'Radiology',
       'Nursing/other'], dtype=object)

In [27]:
discharge.DESCRIPTION.unique()

array(['Report', 'Addendum', 'Nursing Transfer Note', ...,
       'PLACE CATH CAROTID/INOM ART',
       'L US MSK ASPIRATE/INJ GANGLION CYST LEFT',
       'RO HIP NAILING IN OR W/FILMS & FLUORO RIGHT IN O.R.'],
      dtype=object)

In [28]:
print("Number of admissions that have notes are %s"%discharge[discharge.HADM_ID.notnull()].HADM_ID.nunique())

Number of admissions that have notes are 58361


In [29]:
discharge.dropna(subset=["HADM_ID"], inplace=True)

In [30]:
discharge['HADM_ID']=discharge['HADM_ID'].astype(np.int64)
discharge.dtypes

SUBJECT_ID       int64
HADM_ID          int64
CATEGORY        object
DESCRIPTION     object
ISERROR        float64
TEXT            object
dtype: object

In [31]:
discharge[(discharge.CATEGORY=='Discharge summary') & (discharge.HADM_ID==189797) ][['TEXT']]


TEXT
12557  Admission Date:  [**2193-5-11**]       Discharge Date:  [**2193-5-21**]\n\nDate of Birth:   [**2193-5-11**]       Sex:  F\n\nService:  NEONATOLOGY\nHISTORY OF PRESENT ILLNESS:   [**Name (NI) **] [**Known lastname 43327**] is the former 2.32\nkg product of a twin gestation pregnancy, born to a\n30-year-old prima gravida woman.  Prenatal screens:  Blood\ntype A positive, antibody negative, hepatitis B surface\nantigen negative, RPR nonreactive, rubella immune, group beta\ngestation.  Pregnancy was complicated by mild gestational\ndiabetes that was diet controlled.  The mother presented in\nlabor on the day of delivery.  She proceeded to spontaneous\nvaginal delivery.  This twin number two emerged vigorous,\nwith Apgars of 8 at one minute and 9 at five minutes.  She\ndeveloped mild grunting in the delivery room, and was\nadmitted to the Neonatal Intensive Care Unit for treatment of\n\nPHYSICAL EXAMINATION:   Upon admission to the Neonatal\nIntensive Care Unit, weight 2.32 kg (40th percentile), length\n45 cm (30th percentile), head circumference 31 cm (25th\npercentile).  Head, eyes, ears, nose and throat:  Anterior\nfontanel soft and flat, sutures mobile, normal red reflex\nbilaterally, palate intact.  Chest:  Lungs clear and equal,\nmild retractions, intermittent grunting.  Cardiovascular:\nNormal S1, S2, no murmur, normal intensity.  Abdomen:  Soft,\nwith normal bowel sounds, three vessel cord.  Genitourinary:\nNormal female.  Neurologic:  Good tone, alert and responsive.\n\nHOSPITAL COURSE BY SYSTEM:\n1.  Respiratory:  [**Name (NI) **] required treatment for her\nrespiratory distress with continuous positive airway\npressure.  Her maximum oxygen requirement was 30%.  She\nremained on the continuous positive airway pressure through\nday of life number three, when she was changed to nasal\ncannula oxygen.  She required an additional three days of\nnasal cannula oxygen and, by day of life six, was in room\nair.  She remained in room air until discharge.  At the time\nof discharge, her respiratory rates are 30s to 60s, without\nany evidence of distress.  Chest x-ray was consistent with\ntransient tachypnea of the newborn.\n\n2.  Cardiovascular:  [**Name (NI) **] required a normal saline bolus\nshortly after admission for hypotension.  The hypotension\nresolved following the bolus, and she maintained normal blood\npressures through the remainder of the admission.  No murmurs\nhave been noted during the admission.\n\n3.  Fluids, electrolytes and nutrition:  [**Name (NI) **] was initially\nnothing by mouth and maintained on intravenous fluids.  An\ninitial hypoglycemia was resolved with glucose infusion.\nEnteral feeds were started on day of life number two and\ngradually advanced to full volume.  At the time of discharge,\nshe is taking Enfamil 20 or breast feeding ad lib with a\nminimum of 80 cc/kg/day.  Her actual intake has been between\n130 and 150 cc/kg/day.  Discharge weight is 2.23 kg with a\ncorresponding length of 46 cm and head circumference of 32\ncm.  Her low weight occurred on day of life number six at\n2.175 kg.  Serum electrolytes were checked twice in the first\nweek of life and were within normal limits.\n\n4.  Infectious Disease:  Due to the known group beta strep\ncolonization of the mother and her respiratory distress,\n[**Name (NI) **] was evaluated for sepsis.  A white blood cell count was\n9,700, with a differential of 44% polys, 4% bands.  A blood\nculture was obtained prior to starting intravenous ampicillin\nand gentamicin.  Blood culture was no growth at 48 hours, and\nthe antibiotics were discontinued.\n\n5.  Gastrointestinal:  Peak serum bilirubin occurred on day\nof life number four with a total of 5.8/0.3 mg/dl for an\nunconjugated of 5.5 mg/dl.\n\n6.  Neurology:  [**Name (NI) **] had maintained normal neurological\nexamination during the admission.  There were no neurological\nconcerns at the time of discharge.\n\n7.  Audiology:  Hearing screening was performed with\nautomated auditory bra

In [32]:
diag.HADM_ID.max()

199999

In [33]:
discharge.HADM_ID.nunique()

58361

In [34]:
discharge[discharge.HADM_ID==100003]

SUBJECT_ID  HADM_ID           CATEGORY  \
19192         54610   100003  Discharge summary   
76581         54610   100003               Echo   
184869        54610   100003                ECG   
566749        54610   100003            Nursing   
566842        54610   100003         Physician    
566847        54610   100003            Nursing   
566848        54610   100003            Nursing   
566913        54610   100003            Nursing   
567761        54610   100003         Physician    
568203        54610   100003            Nursing   
568288        54610   100003            Nursing   
568289        54610   100003            Nursing   
568332        54610   100003            Nursing   
568360        54610   100003         Physician    
568431        54610   100003         Physician    
568432        54610   100003         Physician    
568435        54610   100003            Nursing   
568735        54610   100003         Physician    
568737        54610   100003         Physician    
568812        54610   100003            Nursing   
568935        54610   100003            Nursing   
569002        54610   100003            Nursing   
569179        54610   100003         Physician    
1069995       54610   100003          Radiology   
1070742       54610   100003          Radiology   
2039175       54610   100003         Physician    

                                       DESCRIPTION  ISERROR  \
19192                                       Report      NaN   
76581                                       Report      NaN   
184869                                      Report      NaN   
566749                       Nursing Transfer Note      NaN   
566842           Physician Resident Admission Note      NaN   
566847                       Nursing Progress Note      NaN   
566848                       Nursing Progress Note      NaN   
566913                       Nursing Progress Note      NaN   
567761   Physician Attending Admission Note - MICU      NaN   
568203                       Nursing Transfer Note      NaN   
568288                       Nursing Progress Note      NaN   
568289                       Nursing Progress Note      NaN   
568332                       Nursing Progress Note      NaN   
568360           Physician Attending Progress Note      NaN   
568431            Physician Resident Progress Note      NaN   
568432            Physician Resident Progress Note      NaN   
568435                       Nursing Transfer Note      NaN   
568735            Physician Resident Progress Note      NaN   
568737            Physician Resident Progress Note      NaN   
568812                       Nursing Progress Note      NaN   
568935                       Nursing Progress Note      NaN   
569002                       Nursing Progress Note      NaN   
569179           Physician Resident Admission Note      NaN   
1069995     LIVER OR GALLBLADDER US (SINGLE ORGAN)      NaN   
1070742                        CHEST (PORTABLE AP)      NaN   
2039175          Physician Attending Progress Note      NaN   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [35]:
discharge_summary=discharge[discharge.CATEGORY=='Discharge summary']

In [36]:
discharge[discharge.CATEGORY=='Nursing'].DESCRIPTION.unique()

array(['Nursing Transfer Note', 'Nursing Progress Note',
       'Nursing discharge note', 'Generic Note',
       'Addendum to transfer Note', 'progress note', 'set as error',
       'Nursing 1900-0700', 'Nursing Note', 'ICU Event Note',
       'Nursing Admission/Progress Note', 'Nursing admit note 2300-0700',
       'ICU Event Note intubation', 'Nursing 0100-0700',
       'NURSING 1900-0700', 'NPN', 'Nursing 2300 - 0700',
       'Nursing Admission Note', 'Nursing admit note.',
       'Nursing admit 2100 - 0700', 'Admission Note', 'Nursing admit',
       'Transfer Note', 'Nursing 0130-0700', 'Nursing Admit Note',
       'Nursing 1900 - 0700', 'Nursing 2000-0700', 'Nuirsing 1900-0700',
       'Cardiology Comprehensive Physician Note', 'nursing 1900-0700',
       'transfer note', 'Nursing  note', 'Progress note',
       'Nursing admit 0001 - 0700', 'ICU Note - CVI',
       'Nursong Progress Note', 'Nursing 1900+-0700', 'Nursing 0300-0700',
       'Nursing 0100 - 0700', 'Nursing',
       '

In [37]:
discharge[discharge.CATEGORY=='Nursing']

SUBJECT_ID  HADM_ID CATEGORY            DESCRIPTION  ISERROR  \
308691        29075   179159  Nursing  Nursing Transfer Note      NaN   
308692        18082   181163  Nursing  Nursing Transfer Note      NaN   
308693        18082   181163  Nursing  Nursing Progress Note      NaN   
308704        18082   181163  Nursing  Nursing Progress Note      NaN   
308705        28742   185325  Nursing  Nursing Transfer Note      NaN   
...             ...      ...      ...                    ...      ...   
2065195        5525   132331  Nursing  Nursing Progress Note      NaN   
2065196       28043   149969  Nursing  Nursing Progress Note      NaN   
2065197       19048   123204  Nursing  Nursing Progress Note      NaN   
2065198        7709   173305  Nursing  Nursing Progress Note      NaN   
2065199       52263   179771  Nursing  Nursing Progress Note      NaN   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [38]:
def textLower(x):
    return x.lower()

In [39]:
def replacenewline(x):
    return(re.sub("\\n"," ",x))
    

In [40]:
def replaceunk(x):
    return re.sub("\[\*\*.*?\*\*\]"," UNK ",x)

In [41]:
def discharge_diagnosis(x):
    y=re.search("discharge diagnosis:",x)
    if y:
        return re.split("discharge diagnosis:",x,1)[1]
    else:
        return None

In [42]:
def final_diagnoses(x):
    y=re.search("final diagnoses:",x)
    if y:
        return re.split("final diagnoses:",x,1)[1]
    else:
        return None

In [43]:
def diagnoses(x):
    y=re.search("final diagnoses:",x)
    
    if y:
        z=re.split("final diagnoses:",x,1)[1]
        a=re.search("discharge diagnosis:",z)
        if a:
            return re.split("discharge diagnosis:",z,1)[1]
        else:
            return z
    else:
        m=re.search("discharge diagnosis:",x)
        if m:
            n=re.split("discharge diagnosis:",x,1)[1]
            o=re.search("final diagnosis:",n)
            if o:
                return re.split("final diagnosis:",n,1)[1]
            else:
                p=re.search("final diagnoses:",n)
                if p:
                    return re.split("final diagnoses:",n,1)[1]
                else:
                    return n
        else:
            return None

In [44]:
discharge.CATEGORY.unique()

array(['Discharge summary', 'Echo', 'ECG', 'Nursing', 'Physician ',
       'Rehab Services', 'Case Management ', 'Respiratory ', 'Nutrition',
       'General', 'Social Work', 'Pharmacy', 'Consult', 'Radiology',
       'Nursing/other'], dtype=object)

In [45]:
discharge.columns

Index(['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'DESCRIPTION', 'ISERROR', 'TEXT'], dtype='object')

In [46]:
discharge[['TEXT']].head(5)


TEXT
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

# <a id="icdpcs">ICD PCS CODES</a>
 Table used are:
 - PROCEDURES_ICD.csv: contains ICD9 codes corresponding to AdmID
 - D_ICD_PROCEDURES.csv: contains ICD9 PCS definitions

In [47]:
procedure=pd.read_csv('Data MIMIC III/PROCEDURES_ICD.csv')

In [48]:
procedure

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331
...,...,...,...,...,...
240090,228330,67415,150871,5,3736
240091,228331,67415,150871,6,3893
240092,228332,67415,150871,7,8872
240093,228333,67415,150871,8,3893


In [95]:
print("Number of records that dont have AdmIds :%d" %procedure[procedure.HADM_ID.isnull()].ROW_ID.count())
print("Number of admIds that dont have ICD codes :%d" %procedure[procedure.ICD9_CODE.isnull()].HADM_ID.count())

Number of records that dont have AdmIds :0
Number of admIds that dont have ICD codes :0


In [50]:
icdproc_freq=procedure.groupby('ICD9_CODE').ICD9_CODE.count().to_frame()
icdproc_freq.rename(columns={icd_freq.columns[0]:'count'},inplace=True)
icdproc_freq.reset_index(level=0,inplace=True)
icdproc_freq


,ICD9_CODE,count
0,10,3
1,11,78
2,12,37
3,13,403
4,14,741
...,...,...
2004,9983,2402
2005,9984,1
2006,9988,4
2007,9995,2


In [52]:
icd9_pcs=pd.read_csv('Data MIMIC III/D_ICD_PROCEDURES.csv')

In [53]:
icd9_pcs

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,264,851,Canthotomy,Canthotomy
1,265,852,Blepharorrhaphy,Blepharorrhaphy
2,266,859,Adjust lid position NEC,Other adjustment of lid position
3,267,861,Lid reconst w skin graft,Reconstruction of eyelid with skin flap or graft
4,268,862,Lid reconst w muc graft,Reconstruction of eyelid with mucous membrane flap or graft
...,...,...,...,...
3877,3344,9959,Vaccination/innocula NEC,Other vaccination and inoculation
3878,3345,9960,Cardiopulm resuscita NOS,"Cardiopulmonary resuscitation, not otherwise specified"
3879,3346,9961,Atrial cardioversion,Atrial cardioversion
3880,3347,9962,Heart countershock NEC,Other electric countershock of heart


In [54]:
icd9_pcs[icd9_pcs.ICD9_CODE==46]

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
61,325,46,Periph nerve transposit,Transposition of cranial and peripheral nerves
358,170,46,Insert 2 vascular stents,Insertion of two vascular stents


In [55]:
icd9_pcs.ICD9_CODE.nunique()

3809

In [56]:
icd9_pcs[icd9_pcs.ICD9_CODE.duplicated()]

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
274,86,64,Perc ins extracran stent,Percutaneous insertion of other extracranial artery stent(s)
276,88,66,PTCA,Percutaneous transluminal coronary angioplasty [PTCA]
277,89,67,Intravas msmnt thorc art,Intravascular pressure measurement of intrathoracic arteries
283,95,73,Rev hip repl-liner/head,"Revision of hip replacement, acetabular liner and/or femoral head only"
298,110,93,Transplant cadaver donor,Transplant from cadaver
...,...,...,...,...
3092,2820,719,Other female genital ops,Other operations on female genital organs
3127,2855,721,Low forceps w episiotomy,Low forceps operation with episiotomy
3235,2963,729,Instrument delivery NOS,Unspecified instrumental delivery
3399,3479,863,Other local destruc skin,Other local excision or destruction of lesion or tissue of skin and subcutaneous tissue


In [57]:
icdproc_freq.sort_values(by='count',ascending=False).head(100).reset_index(drop=True)

,ICD9_CODE,count
0,3893,14731
1,9604,10333
2,966,9300
3,9671,9100
4,9904,7244
...,...,...
95,3972,403
96,13,403
97,3892,399
98,8162,395


In [97]:
listoffreqpcs=icdproc_freq.sort_values(by='count',ascending=False).head(100).ICD9_CODE.to_list()
listoffreqpcs

[3893,
 9604,
 966,
 9671,
 9904,
 3961,
 9672,
 9955,
 8856,
 3891,
 3615,
 9915,
 8872,
 3722,
 3324,
 3995,
 4513,
 9390,
 3723,
 9983,
 5491,
 331,
 640,
 8853,
 9907,
 3612,
 3491,
 3895,
 311,
 9920,
 3404,
 3897,
 40,
 8841,
 4311,
 3613,
 3521,
 66,
 3322,
 3606,
 9905,
 3607,
 3323,
 3601,
 8964,
 3761,
 3611,
 45,
 8855,
 17,
 9962,
 4523,
 8842,
 387,
 9605,
 3950,
 14,
 8847,
 8659,
 9910,
 8622,
 5459,
 4443,
 159,
 9960,
 8848,
 4516,
 3721,
 8852,
 9607,
 9925,
 3845,
 9929,
 3522,
 131,
 5187,
 3321,
 3990,
 22,
 309,
 3778,
 4639,
 3783,
 5122,
 3772,
 5185,
 3734,
 4562,
 46,
 3614,
 3726,
 370,
 3512,
 3979,
 4131,
 3972,
 13,
 3892,
 8162,
 9359]

> Below dictionary of ICD9 PCS codes to ICD10 codes is converted through <https://icd.codes/convert/icd9-to-icd10-pcs>

In [58]:
icd10_pcs_dict={'ICD9_CODE':[9604,9604,966,966,966,9671,9671,3961,9672,9955,9955,9915,9915,9915,9915,3722,3722,3995,3995,4513,9390,9390,9390,3723,3723,9983,640,640,640,3491,3491,3895,3895,3404,3404,3404,3404,4311,4311,3322,3322,3322,9905,9905,9905,9905,3323,3323,3323,3761,3761,9962,4523,9605,9605,9605,9605,9910,9910,9910,9910,9910,4443,159,159,159,159,9960,3721,3721,9607,9607,9925,9925,9925,9925,3522,3522,3321,3778,3778,3783,3783,3783,3783,5122,3722,3722,5185,5185,3726,3726,3726,3512,3512,3512,3512,3512,3892],
                'icd10_pcs':['0BH.17EZ','0BH.18EZ','0DH.67UZ','0DH.68UZ','3E0.G36Z','5A1.935Z','5A1.945Z','5A1.221Z','5A1.955Z','3E0.134Z','3E0.234Z','3E0.336Z','3E0.436Z','3E0.536Z','3E0.636Z','4A0.20N7','4A0.23N7','5A1.D00Z','5A1.D60Z','0DJ.08ZZ','5A0.9357','5A0.9457','5A0.9557','4A0.20N8','4A0.23N8','6A6.00ZZ','0VT.T0ZZ','0VT.T4ZZ','0VT.TXZZ','0W9.93ZZ','0W9.B3ZZ','05H.Y33Z','06H.Y33Z','0W9.930Z','0W9.940Z','0W9.B30Z','0W9.B40Z','0DH.63UZ','0DH.64UZ','0BJ.08ZZ','0BJ.K8ZZ','0BJ.L8ZZ','302.33R1','302.43R1','302.53R1','302.63R1','0BJ.08ZZ','0BJ.K8ZZ','0BJ.L8ZZ',
                            '5A0.2110','5A0.2210','5A2.204Z','0DJ.D8ZZ','0B7.17DZ','0B7.18DZ','0BH.07DZ','0WH.Q7YZ','3E0.3317','3E0.4317','3E0.5317','3E0.6317','3E0.8317','0W3.P8ZZ','08Q.L0ZZ','08Q.L3ZZ','08Q.M0ZZ','08Q.M3ZZ','5A1.2012','4A0.20N6','4A0.23N6','0D9.670Z','0D9.680Z','3E0.3305','3E0.4305','XW0.3351','XW0.4351','02R.F0JZ','02R.F4JZ','0BJ.08ZZ','5A1.213Z','5A1.223Z','0JH.606Z','0JH.636Z','0JH.806Z','0JH.836Z','0FT.40ZZ','4A0.20N7','4A0.23N7','0F9.C80Z','0F9.C8ZZ','3E0.53KZ','3E0.63KZ','4A0.23FZ','027.G04Z','027.G0DZ','027.G0ZZ','02N.G0ZZ','02Q.G0ZZ','06H.033T'],
                'long_title':['Insertion of Endotracheal Airway into Trachea, Via Natural or Artificial Opening','Insertion of Endotracheal Airway into Trachea, Via Natural or Artificial Opening Endoscopic','Insertion of Feeding Device into Stomach, Via Natural or Artificial Opening','Insertion of Feeding Device into Stomach, Via Natural or Artificial Opening Endoscopic','Introduction of Nutritional Substance into Upper GI, Percutaneous Approach','Respiratory Ventilation, Less than 24 Consecutive Hours','Respiratory Ventilation, 24-96 Consecutive Hours',
                             'Performance of Cardiac Output, Continuous','Respiratory Ventilation, Greater than 96 Consecutive Hours','Introduction of Serum, Toxoid and Vaccine into Subcutaneous Tissue, Percutaneous Approach','Introduction of Serum, Toxoid and Vaccine into Muscle, Percutaneous Approach','Introduction of Nutritional Substance into Peripheral Vein, Percutaneous Approach','Introduction of Nutritional Substance into Central Vein, Percutaneous Approach','Introduction of Nutritional Substance into Peripheral Artery, Percutaneous Approach','Introduction of Nutritional Substance into Central Artery, Percutaneous Approach',
                             'Measurement of Cardiac Sampling and Pressure, Left Heart, Open Approach','Measurement of Cardiac Sampling and Pressure, Left Heart, Percutaneous Approach','Performance of Urinary Filtration, Single','Performance of Urinary Filtration, Multiple','Inspection of Upper Intestinal Tract, Via Natural or Artificial Opening Endoscopic','Assistance with Respiratory Ventilation, Less than 24 Consecutive Hours, Continuous Positive Airway Pressure','Assistance with Respiratory Ventilation, 24-96 Consecutive Hours, Continuous Positive Airway Pressure',
                             'Assistance with Respiratory Ventilation, Greater than 96 Consecutive Hours, Continuous Positive Airway Pressure','Measurement of Cardiac Sampling and Pressure, Bilateral, Open Approach','Measurement of Cardiac Sampling and Pressure, Bilateral, Percutaneous Approach','Phototherapy of Skin, Single','Resection of Prepuce, Open Approach','Resection of Prepuce, Percutaneous Endoscopic Approach','Resection of Prepuce, External Approach','Drainage of Right Pleural Cavity, Percutaneous Approach','Drainage of Left Pleural Cavity, Percutaneous Approach','Insertion of Infusion Device into Upper Vein, Percutaneous Approach',
                             'Insertion of Infusion Device into Lower Vein, Percutaneous Approach','Drainage of Right Pleural Cavity with Drainage Device, Percutaneous Approach','Drainage of Right Pleural Cavity with Drainage Device, Percutaneous Endoscopic Approach','Drainage of Left Pleural Cavity with Drainage Device, Percutaneous Approach',' Drainage of Left Pleural Cavity with Drainage Device, Percutaneous Endoscopic Approach','nsertion of Feeding Device into Stomach, Percutaneous Approach','Insertion of Feeding Device into Stomach, Percutaneous Endoscopic Approach','Inspection of Tracheobronchial Tree, Via Natural or Artificial Opening Endoscopic',
                             'Inspection of Right Lung, Via Natural or Artificial Opening Endoscopic','Inspection of Left Lung, Via Natural or Artificial Opening Endoscopic','Transfusion of Nonautologous Platelets into Peripheral Vein, Percutaneous Approach','Transfusion of Nonautologous Platelets into Central Vein, Percutaneous Approach','Transfusion of Nonautologous Platelets into Peripheral Artery, Percutaneous Approach','Transfusion of Nonautologous Platelets into Central Artery, Percutaneous Approach','Inspection of Tracheobronchial Tree, Via Natural or Artificial Opening Endoscopic',
                             'Inspection of Right Lung, Via Natural or Artificial Opening Endoscopic','Inspection of Left Lung, Via Natural or Artificial Opening Endoscopic','Assistance with Cardiac Output using Balloon Pump, Intermittent','Assistance with Cardiac Output using Balloon Pump, Continuous','Restoration of Cardiac Rhythm, Single','Inspection of Lower Intestinal Tract, Via Natural or Artificial Opening Endoscopic','Dilation of Trachea with Intraluminal Device, Via Natural or Artificial Opening','Dilation of Trachea with Intraluminal Device, Via Natural or Artificial Opening Endoscopic',
                             'Insertion of Intraluminal Device into Tracheobronchial Tree, Via Natural or Artificial Opening','Insertion of Other Device into Respiratory Tract, Via Natural or Artificial Opening','Introduction of Other Thrombolytic into Peripheral Vein, Percutaneous Approach','Introduction of Other Thrombolytic into Central Vein, Percutaneous Approach','Introduction of Other Thrombolytic into Peripheral Artery, Percutaneous Approach','Introduction of Other Thrombolytic into Central Artery, Percutaneous Approach','Introduction of Other Thrombolytic into Heart, Percutaneous Approach',
                             'Control Bleeding in Gastrointestinal Tract, Via Natural or Artificial Opening Endoscopic','Repair Right Extraocular Muscle, Open Approach','Repair Right Extraocular Muscle, Percutaneous Approach','Repair Left Extraocular Muscle, Open Approach','Repair Left Extraocular Muscle, Percutaneous Approach','Performance of Cardiac Output, Single, Manual','Measurement of Cardiac Sampling and Pressure, Right Heart, Open Approach','Measurement of Cardiac Sampling and Pressure, Right Heart, Percutaneous Approach','Drainage of Stomach with Drainage Device, Via Natural or Artificial Opening',
                             'Drainage of Stomach with Drainage Device, Via Natural or Artificial Opening Endoscopic','Introduction of Other Antineoplastic into Peripheral Vein, Percutaneous Approach','Introduction of Other Antineoplastic into Central Vein, Percutaneous Approach','Introduction of Blinatumomab Antineoplastic Immunotherapy into Peripheral Vein, Percutaneous Approach, New Technology Group 1','Introduction of Blinatumomab Antineoplastic Immunotherapy into Central Vein, Percutaneous Approach, New Technology Group 1','Replacement of Aortic Valve with Synthetic Substitute, Open Approach',
                             'Replacement of Aortic Valve with Synthetic Substitute, Percutaneous Endoscopic Approach','Inspection of Tracheobronchial Tree, Via Natural or Artificial Opening Endoscopic','Performance of Cardiac Pacing, Intermittent','Performance of Cardiac Pacing, Continuous','Insertion of Pacemaker, Dual Chamber into Chest Subcutaneous Tissue and Fascia, Open Approach','Insertion of Pacemaker, Dual Chamber into Chest Subcutaneous Tissue and Fascia, Percutaneous Approach','Insertion of Pacemaker, Dual Chamber into Abdomen Subcutaneous Tissue and Fascia, Open Approach','Insertion of Pacemaker, Dual Chamber into Abdomen Subcutaneous Tissue and Fascia, Percutaneous Approach',
                             'Resection of Gallbladder, Open Approach','Measurement of Cardiac Sampling and Pressure, Left Heart, Open Approach','Measurement of Cardiac Sampling and Pressure, Left Heart, Percutaneous Approach','Drainage of Ampulla of Vater with Drainage Device, Via Natural or Artificial Opening Endoscopic','Drainage of Ampulla of Vater, Via Natural or Artificial Opening Endoscopic','Introduction of Other Diagnostic Substance into Peripheral Artery, Percutaneous Approach','Introduction of Other Diagnostic Substance into Central Artery, Percutaneous Approach','Measurement of Cardiac Rhythm, Percutaneous Approach',
                             'Dilation of Mitral Valve with Drug-eluting Intraluminal Device, Open Approach','Dilation of Mitral Valve with Intraluminal Device, Open Approach','Dilation of Mitral Valve, Open Approach','Release Mitral Valve, Open Approach','Repair Mitral Valve, Open Approach','Insertion of Infusion Device, Via Umbilical Vein, into Inferior Vena Cava, Percutaneous Approach']}

In [59]:
icd10_pcs=pd.DataFrame.from_dict(icd10_pcs_dict)

In [60]:
icd10_pcs

,ICD9_CODE,icd10_pcs,long_title
0,9604,0BH.17EZ,"Insertion of Endotracheal Airway into Trachea, Via Natural or Artificial Opening"
1,9604,0BH.18EZ,"Insertion of Endotracheal Airway into Trachea, Via Natural or Artificial Opening Endoscopic"
2,966,0DH.67UZ,"Insertion of Feeding Device into Stomach, Via Natural or Artificial Opening"
3,966,0DH.68UZ,"Insertion of Feeding Device into Stomach, Via Natural or Artificial Opening Endoscopic"
4,966,3E0.G36Z,"Introduction of Nutritional Substance into Upper GI, Percutaneous Approach"
...,...,...,...
94,3512,027.G0DZ,"Dilation of Mitral Valve with Intraluminal Device, Open Approach"
95,3512,027.G0ZZ,"Dilation of Mitral Valve, Open Approach"
96,3512,02N.G0ZZ,"Release Mitral Valve, Open Approach"
97,3512,02Q.G0ZZ,"Repair Mitral Valve, Open Approach"


> ICD10 pcs code that have more than 1 ICD9 codes

In [104]:
duplicate_icd10pcs=icd10_pcs[icd10_pcs.duplicated(['icd10_pcs'])].icd10_pcs.to_list()
duplicate_icd10pcs

['0BJ.08ZZ', '0BJ.K8ZZ', '0BJ.L8ZZ', '0BJ.08ZZ', '4A0.20N7', '4A0.23N7']

In [105]:
icd10_pcs[icd10_pcs['icd10_pcs'].isin(duplicate_icd10pcs)]

,ICD9_CODE,icd10_pcs,long_title
15,3722,4A0.20N7,"Measurement of Cardiac Sampling and Pressure, Left Heart, Open Approach"
16,3722,4A0.23N7,"Measurement of Cardiac Sampling and Pressure, Left Heart, Percutaneous Approach"
39,3322,0BJ.08ZZ,"Inspection of Tracheobronchial Tree, Via Natural or Artificial Opening Endoscopic"
40,3322,0BJ.K8ZZ,"Inspection of Right Lung, Via Natural or Artificial Opening Endoscopic"
41,3322,0BJ.L8ZZ,"Inspection of Left Lung, Via Natural or Artificial Opening Endoscopic"
46,3323,0BJ.08ZZ,"Inspection of Tracheobronchial Tree, Via Natural or Artificial Opening Endoscopic"
47,3323,0BJ.K8ZZ,"Inspection of Right Lung, Via Natural or Artificial Opening Endoscopic"
48,3323,0BJ.L8ZZ,"Inspection of Left Lung, Via Natural or Artificial Opening Endoscopic"
78,3321,0BJ.08ZZ,"Inspection of Tracheobronchial Tree, Via Natural or Artificial Opening Endoscopic"
86,3722,4A0.20N7,"Measurement of Cardiac Sampling and Pressure, Left Heart, Open Approach"


> List of ICD9 code that more than 5 ICD10 PCS correspondence and left out

In [108]:
leftoutpcs=[item for item in listoffreqpcs if item not in icd10_pcs.ICD9_CODE.to_list()]
leftoutpcs

59

In [109]:
adm_icd_pcs=pd.merge(procedure[['HADM_ID','ICD9_CODE']],icd10_pcs[['ICD9_CODE','icd10_pcs']],how='left',on='ICD9_CODE')

In [110]:
adm_icd_pcs

,HADM_ID,ICD9_CODE,icd10_pcs
0,154460,3404,0W9.930Z
1,154460,3404,0W9.940Z
2,154460,3404,0W9.B30Z
3,154460,3404,0W9.B40Z
4,130856,9671,5A1.935Z
...,...,...,...
357296,150871,3893,NaN
357297,150871,3722,4A0.20N7
357298,150871,3722,4A0.23N7
357299,150871,3722,4A0.20N7


In [111]:
adm_icd_pcs[adm_icd_pcs.icd10_pcs.isna()]

,HADM_ID,ICD9_CODE,icd10_pcs
6,130856,3893,NaN
8,119355,331,NaN
9,119355,3893,NaN
10,158060,34,NaN
11,189217,3613,NaN
...,...,...,...
357292,150871,3727,NaN
357293,150871,3736,NaN
357294,150871,3893,NaN
357295,150871,8872,NaN


In [112]:
adm_icd_pcs['icd10_pcs']=adm_icd_pcs['icd10_pcs'].fillna('Misc')

In [113]:
adm_icdpcs_agg=adm_icd_pcs.groupby('HADM_ID').agg({'ICD9_CODE':lambda x:','.join(map(str,x)),'icd10_pcs':lambda y: ','.join(y)}).reset_index()

In [114]:
adm_icdpcs_agg.head(5)

,HADM_ID,ICD9_CODE,icd10_pcs
0,100003,"4443,9607,9607,9904,3893","0W3.P8ZZ,0D9.670Z,0D9.680Z,Misc,Misc"
1,100006,"9390,9390,9390,9925,9925,9925,9925","5A0.9357,5A0.9457,5A0.9557,3E0.3305,3E0.4305,XW0.3351,XW0.4351"
2,100007,"4562,5459","Misc,Misc"
3,100009,"3613,3615,3795,3961","Misc,Misc,Misc,5A1.221Z"
4,100010,"5551,540,403","Misc,Misc,Misc"
